In [81]:
import numpy as np

def select_dfc_indices(regions_mask, region_label, dfc_array):
    
    # 1. Flatten the regions mask so every pixel is on the same axis
    flattened_regions_mask = regions_mask.flatten()
    flattened_regions_mask = flattened_regions_mask[~np.isnan(flattened_regions_mask)] # Remove nan values

    # 2. Determine which pixels should be kept
    keep_indices = np.where(flattened_regions_mask==region_label)[0]

    # 3. Take the shape of the provided dFC array (first frame) and put it back in square form
    N = int((np.sqrt(8*dfc_array.shape[1]+1)-1)/2)
    mask = flat_to_symmetric(np.zeros_like(dfc_array[0]), N)

    # 4. Make a boolean mask with the given pixels
    mask = np.asarray(mask, dtype=bool)
    for index in keep_indices:
        mask[index,keep_indices] = True

    # 5. Flatten the mask by keeping only the upper triangle and diagonal, in the same way the dFC is flattened
    flattened_mask = mask[np.triu_indices(mask.shape[0])]

    # 6. Slice the dFC
    sliced_dfc = dfc_array[:,flattened_mask]

    return sliced_dfc

def flat_to_symmetric(flat, N):
    """Convert a flattened upper triangle vector to a full symmetric matrix."""
    mat = np.zeros((N, N))
    inds = np.triu_indices(N)
    mat[inds] = flat
    mat[(inds[1], inds[0])] = flat  # Reflect upper triangle to lower
    return mat


def compute_dfc_flat(signals):
    T, N = signals.shape
    # print(f"{T}, {N}")
    triu_len = (N * (N + 1)) // 2  # Number of upper triangle elements (including diagonal)
    dFC_flat = np.zeros((T, triu_len))

    for t in range(T):
        idx = 0
        for i in range(N):
            for j in range(i, N):
                dFC_flat[t, idx] = signals[t, i] * signals[t, j]
                idx += 1

    return dFC_flat

def format_array(array, return_mask=False, zscore_signal=False):
    """Format a 3D array prior to computing FC.
    """

    array = array.reshape(array.shape[0], -1) # Reshapes to a time x nodes matrix
    mask = ~np.isnan(array[0,:])
    array = np.array([row[~np.isnan(row)] for row in array])

    if zscore_signal:
        for i in range(array.shape[1]):
            array[:,i] = zscore(array[:,i])

    if return_mask: return array, mask
    return array

def select_region_pixels(regions_mask, region_label, data):
    """Given a region mask, keeps only pixels in data corresponding to a specific region label.
    The rest is converted to np.nan and the array is resized to the smallest array possible containing
    all non-nan values.

    Args:
        regions_mask (np.array): 2d array containing the regions labels
        region_label (int): Region label to keep in the data
        data (np.array): 2d or 3d array to filter using the specified regions mask
    """

    bool_mask = (regions_mask == region_label) # True where the regions correspond with the given index
    masked_data = data*np.where(bool_mask==False, np.nan, 1) # Change values to nan where the mask is False

    # Crop data to smallest bounding box
    valid_rows = np.any(~np.isnan(masked_data[0]), axis=1)  # Check for valid values along height (axis 1)
    valid_cols = np.any(~np.isnan(masked_data[0]), axis=0)  # Check for valid values along width (axis 0)
    

    # Crop data to the smallest bounding box in spatial dimensions (height, width) using the mask
    if data.ndim == 2:
        cropped_data = masked_data[valid_rows, :][:, valid_cols]
    elif data.ndim == 3:
        cropped_data = masked_data[:, valid_rows, :][:, :, valid_cols]
    
    return cropped_data

In [82]:
# 1. Make fake data

data = np.array([[[1, 2, 3], [4, 5, np.nan], [7, np.nan, 9]],
                 [[11, 12, 13], [14, 15, np.nan], [17, np.nan, 19]]])
print(data)

[[[ 1.  2.  3.]
  [ 4.  5. nan]
  [ 7. nan  9.]]

 [[11. 12. 13.]
  [14. 15. nan]
  [17. nan 19.]]]


In [83]:
# 2. Define a regions mask based on the original 3x3 shape

print("Regions mask:")
regions_mask = np.array([[1, 1, 2], [1, 2, 2], [np.nan, 3, 3]])
print(regions_mask)

print("\nData:")
print(data)

print(f"\nIf we select region 1, the mask should select elements [1, 2, 4] and [11, 12, 14].")
print(f"If we select region 2, the mask should select elements [3, 5, (np.nan)] and [13, 15, (np.nan)].")
print(f"If we select region 3, the mask should select elements [(np.nan), 9] and [(np.nan), 19].")



# Remove the mask elements where there are nan elements in the original data
remove_indices = np.where(np.isnan(data[0])) # Where there are nans in the first frame of signals
regions_mask[remove_indices] = np.nan

print(f"\nNew regions mask:\n{regions_mask}")

Regions mask:
[[ 1.  1.  2.]
 [ 1.  2.  2.]
 [nan  3.  3.]]

Data:
[[[ 1.  2.  3.]
  [ 4.  5. nan]
  [ 7. nan  9.]]

 [[11. 12. 13.]
  [14. 15. nan]
  [17. nan 19.]]]

If we select region 1, the mask should select elements [1, 2, 4] and [11, 12, 14].
If we select region 2, the mask should select elements [3, 5, (np.nan)] and [13, 15, (np.nan)].
If we select region 3, the mask should select elements [(np.nan), 9] and [(np.nan), 19].

New regions mask:
[[ 1.  1.  2.]
 [ 1.  2. nan]
 [nan nan  3.]]


In [84]:
# 3. Test the pixel selection on this data

pix = select_region_pixels(regions_mask, region_label=2, data=data)

print(pix)

print("\nThe function succesfully selects the right pixels and crops to the smallest bounding array.")

[[[nan  3.]
  [ 5. nan]]

 [[nan 13.]
  [15. nan]]]

The function succesfully selects the right pixels and crops to the smallest bounding array.


In [85]:
# 4. Compute dFC from the data

flattened_data, elements_mask = format_array(data, return_mask=True)
print(f"Flattened data without nans:\n{flattened_data}")

dfc = compute_dfc_flat(flattened_data)
print(f"\nFlat upper triangle dfc:\n{dfc}")

N = int((np.sqrt(8*dfc.shape[1]+1)-1)/2)
first_fc_frame = flat_to_symmetric(dfc[0,:], N)
print(f"\nFirst FC frame:\n{first_fc_frame}")

Flattened data without nans:
[[ 1.  2.  3.  4.  5.  7.  9.]
 [11. 12. 13. 14. 15. 17. 19.]]

Flat upper triangle dfc:
[[  1.   2.   3.   4.   5.   7.   9.   4.   6.   8.  10.  14.  18.   9.
   12.  15.  21.  27.  16.  20.  28.  36.  25.  35.  45.  49.  63.  81.]
 [121. 132. 143. 154. 165. 187. 209. 144. 156. 168. 180. 204. 228. 169.
  182. 195. 221. 247. 196. 210. 238. 266. 225. 255. 285. 289. 323. 361.]]

First FC frame:
[[ 1.  2.  3.  4.  5.  7.  9.]
 [ 2.  4.  6.  8. 10. 14. 18.]
 [ 3.  6.  9. 12. 15. 21. 27.]
 [ 4.  8. 12. 16. 20. 28. 36.]
 [ 5. 10. 15. 20. 25. 35. 45.]
 [ 7. 14. 21. 28. 35. 49. 63.]
 [ 9. 18. 27. 36. 45. 63. 81.]]


Knowing that selecting region 1 would select elements [1, 2, 4] on the first frame, we should obtain the following reduced dfc matrix:

In [86]:
x = np.array([[1, 2, 4], [2, 4, 8], [4, 8, 16]])
print(x)

[[ 1  2  4]
 [ 2  4  8]
 [ 4  8 16]]


In [87]:
# Test if we obtain the right dfc matrix

sliced_dfc = select_dfc_indices(regions_mask, region_label=1, dfc_array=dfc)

print(sliced_dfc)

[[  1.   2.   4.   4.   8.  16.]
 [121. 132. 154. 144. 168. 196.]]


We do obtain the right dfc in flattened form. Now if we wanted to isolate the interaction between two regions, we would need to slice the array differently on both axes. For instance, if we wanted the interaction between region 1 (elements [1, 2, 4]) and region 2 (elements [3, 5]), we would obtain

In [88]:
x = np.array([[3, 6, 12], [5, 10, 20]])
print(x)

[[ 3  6 12]
 [ 5 10 20]]


Notably, this is not symmetrical because we look for two different regions. We would need to modify the function like so:

In [ ]:
def select_dfc_indices(regions_mask, region_labels, dfc_array):

    if isinstance(region_labels, tuple):
        region_a, region_b = region_labels
    else:
        region_a = region_labels
        region_b = region_labels
    
    # 1. Flatten the regions mask so every pixel is on the same axis
    flattened_regions_mask = regions_mask.flatten()
    flattened_regions_mask = flattened_regions_mask[~np.isnan(flattened_regions_mask)] # Remove nan values

    # 2. Determine which pixels should be kept
    keep_indices_a = np.where(flattened_regions_mask==region_a)[0]
    print(keep_indices_a)
    keep_indices_b = np.where(flattened_regions_mask==region_b)[0]
    print(keep_indices_b)

    # 3. Take the shape of the provided dFC array (first frame) and put it back in square form
    N = int((np.sqrt(8*dfc_array.shape[1]+1)-1)/2)
    mask = flat_to_symmetric(np.zeros_like(dfc_array[0]), N)

    # 4. Make a boolean mask with the given pixels
    mask = np.asarray(mask, dtype=bool)
    for index_a in keep_indices_a:
        mask[index_a,keep_indices_b] = True
    for index_b in keep_indices_b: # Reflected to account for different regions; only the upper triangle will be considered in the end anyway
        mask[index_b,keep_indices_a] = True

    # 5. Make the mask symmetric by reflecting the lower triangle on the upper

    # 5. Flatten the mask by keeping only the upper triangle and diagonal, in the same way the dFC is flattened
    flattened_mask = mask[np.triu_indices(mask.shape[0])]

    # 6. Slice the dFC
    sliced_dfc = dfc_array[:,flattened_mask]

    return sliced_dfc

sliced_dfc = select_dfc_indices(regions_mask, region_labels=(1,2), dfc_array=dfc)

print(sliced_dfc)

[0 1 3]
[2 4]
[[  3.   5.   6.  10.  12.  20.]
 [143. 165. 156. 180. 182. 210.]]


We do obtain the flattened array, though transposed from our prediction. Note that it will *always* return the array with the columns selected from the second region provided (region 2 here) and the rows from the first region provided (region 1).